<a href="https://colab.research.google.com/github/Minwoo-study/Naver_map_rating/blob/main/FINAL_KcELECTRA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pytorch + HuggingFace 
# KcElectra Model
**KcELECTRA**-Base FineTune with PyTorch-Lightning1.3 사용 <br>
https://www.kaggle.com/code/junbumlee/kcelectra-base-finetune-with-pytorch-lightning1-3/notebook

## Datasets
- 지역 20곳 & 각 지역별 식당 40여개 리뷰 전체 크롤링 <br>
- 강릉, 건대, 경주, 광안리, 남포동, 부평, 사당, 서귀포, 서면, 속초, 송도, 연신내, 왕십리, 일산, 전주한옥마을, 제주, 판교, 해운대, 혜화, 홍대

## References
- https://huggingface.co/transformers/training.html
- https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html
- https://tutorials.pytorch.kr/beginner/blitz/cifar10_tutorial.html
- https://wikidocs.net/44249

## 주의사항
- GPU로 돌려주세요

#Preparation

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 설치
!pip install transformers # HuggingFace transformers
!pip install pandas tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 25.9 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW, TextClassificationPipeline
from tqdm.notebook import tqdm

In [ ]:
# 데이터셋 불러오기
train = './train_set.csv' 
test = './test_set.csv'
test_pd = pd.read_csv('./test_set.csv')

In [ ]:
test_pd

,리뷰,평점,라벨링
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,1
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,1
2,좋아요,5.0,1
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,1
4,귀엽다,3.0,0
...,...,...,...
27924,생각보다 그냥 그래욤,3.5,0
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,1
27926,친절해요,5.0,1
27927,좋아요,5.0,1


In [ ]:
# GPU 사용
device = torch.device("cuda")

# Dataset

In [ ]:
class ClassifyDataset(Dataset):
  
  def __init__(self, csv_file):
    # csv 읽어오기 & NaN 처리
    self.dataset = pd.read_csv(csv_file).dropna(axis=0) 
    # 중복 제거
    self.dataset.drop_duplicates(subset=['리뷰'], inplace=True)
    # pretrained model
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base") 

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:3].values # idx 행과 0,1,2 columns
    text = row[0]
    y = row[2]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt', # return pytorch tensors
        truncation=True, # reducing long sequences
        max_length=256,
        pad_to_max_length=True, # padding
        add_special_tokens=True # 문장 앞뒤로 special tocken - padding 부착
        )
    
    input_ids = inputs['input_ids'][0] 
    attention_mask = inputs['attention_mask'][0] # the position of the padded indices so that the model does not attend to them

    return input_ids, attention_mask, y

In [ ]:
train_dataset = ClassifyDataset(train)
test_dataset = ClassifyDataset(test)

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/504 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/124 [00:00<?, ?B/s]

                 평점           라벨링
count  24689.000000  24689.000000
mean       3.717425      0.547005
std        1.400143      0.497796
min        0.500000      0.000000
25%        3.000000      0.000000
50%        4.000000      1.000000
75%        5.000000      1.000000
max        5.000000      1.000000
                 평점           라벨링
count  22951.000000  22951.000000
mean       4.484619      0.871378
std        0.939953      0.334788
min        0.500000      0.000000
25%        4.000000      1.000000
50%        5.000000      1.000000
75%        5.000000      1.000000
max        5.000000      1.000000


# Create Model & Train

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("beomi/KcELECTRA-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model,device=0) # gpu로 변환

Downloading:   0%|          | 0.00/475M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

In [ ]:
epochs = 5
batch_size = 16

optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
losses = []
accuracies = []

for i in range(epochs): # epoch 5
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train() # forward

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader): # tqdm 진행상황 확인 
    optimizer.zero_grad()
  
    y_batch = y_batch.type(torch.LongTensor)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0] 
    loss = F.cross_entropy(y_pred, y_batch)

    loss.backward()
    optimizer.step() 

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total) 

  0%|          | 0/1544 [00:00<?, ?it/s]

D:\Anaconda3\envs\pytorch\lib\site-packages\transformers\tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 64.4806479215622 Accuracy: tensor(0.6850, device='cuda:0')
Batch Loss: 107.01671670377254 Accuracy: tensor(0.7775, device='cuda:0')
Batch Loss: 143.34920819103718 Accuracy: tensor(0.8075, device='cuda:0')
Batch Loss: 179.03383566439152 Accuracy: tensor(0.8225, device='cuda:0')
Batch Loss: 212.1776396408677 Accuracy: tensor(0.8325, device='cuda:0')
Batch Loss: 246.73247396945953 Accuracy: tensor(0.8371, device='cuda:0')
Batch Loss: 278.6070186495781 Accuracy: tensor(0.8423, device='cuda:0')
Batch Loss: 310.52736485749483 Accuracy: tensor(0.8470, device='cuda:0')
Batch Loss: 342.0600197315216 Accuracy: tensor(0.8494, device='cuda:0')
Batch Loss: 372.9753346890211 Accuracy: tensor(0.8521, device='cuda:0')
Batch Loss: 404.70112723857164 Accuracy: tensor(0.8539, device='cuda:0')
Batch Loss: 435.95427387207747 Accuracy: tensor(0.8557, device='cuda:0')
Batch Loss: 467.7527089305222 Accuracy: tensor(0.8567, device='cuda:0')
Batch Loss: 499.15058648213744 Accuracy: tensor(0.8573, de

  0%|          | 0/1544 [00:00<?, ?it/s]

Batch Loss: 28.614685118198395 Accuracy: tensor(0.8900, device='cuda:0')
Batch Loss: 55.373286709189415 Accuracy: tensor(0.8887, device='cuda:0')
Batch Loss: 83.47730034589767 Accuracy: tensor(0.8879, device='cuda:0')
Batch Loss: 111.16880089044571 Accuracy: tensor(0.8870, device='cuda:0')
Batch Loss: 139.56534191966057 Accuracy: tensor(0.8868, device='cuda:0')
Batch Loss: 167.64895414188504 Accuracy: tensor(0.8863, device='cuda:0')
Batch Loss: 194.24426377937198 Accuracy: tensor(0.8875, device='cuda:0')
Batch Loss: 222.10936192050576 Accuracy: tensor(0.8873, device='cuda:0')
Batch Loss: 251.4648222438991 Accuracy: tensor(0.8868, device='cuda:0')
Batch Loss: 276.43356608971953 Accuracy: tensor(0.8884, device='cuda:0')
Batch Loss: 304.2497856132686 Accuracy: tensor(0.8877, device='cuda:0')
Batch Loss: 333.5603560619056 Accuracy: tensor(0.8867, device='cuda:0')
Batch Loss: 360.01112350076437 Accuracy: tensor(0.8868, device='cuda:0')
Batch Loss: 386.86097927019 Accuracy: tensor(0.8868, de

...

Batch Loss: 24.612358547747135 Accuracy: tensor(0.9044, device='cuda:0')
Batch Loss: 49.819290675222874 Accuracy: tensor(0.9028, device='cuda:0')
Batch Loss: 74.34831061959267 Accuracy: tensor(0.9015, device='cuda:0')
Batch Loss: 98.72730503231287 Accuracy: tensor(0.9030, device='cuda:0')
Batch Loss: 122.8496465254575 Accuracy: tensor(0.9033, device='cuda:0')
Batch Loss: 150.31846051104367 Accuracy: tensor(0.9005, device='cuda:0')
Batch Loss: 172.20122371427715 Accuracy: tensor(0.9031, device='cuda:0')
Batch Loss: 197.35280418209732 Accuracy: tensor(0.9019, device='cuda:0')
Batch Loss: 220.97426532022655 Accuracy: tensor(0.9022, device='cuda:0')
Batch Loss: 245.73962276615202 Accuracy: tensor(0.9010, device='cuda:0')
Batch Loss: 272.1496545765549 Accuracy: tensor(0.9005, device='cuda:0')
Batch Loss: 295.55024489946663 Accuracy: tensor(0.9004, device='cuda:0')
Batch Loss: 323.32320905663073 Accuracy: tensor(0.8989, device='cuda:0')
Batch Loss: 346.72793690674007 Accuracy: tensor(0.8992,

  0%|          | 0/1544 [00:00<?, ?it/s]

Batch Loss: 22.860377326607704 Accuracy: tensor(0.9031, device='cuda:0')
Batch Loss: 43.852607157081366 Accuracy: tensor(0.9100, device='cuda:0')
Batch Loss: 65.79034464433789 Accuracy: tensor(0.9096, device='cuda:0')
Batch Loss: 88.3929106388241 Accuracy: tensor(0.9095, device='cuda:0')
Batch Loss: 110.54678423516452 Accuracy: tensor(0.9111, device='cuda:0')
Batch Loss: 131.4700116943568 Accuracy: tensor(0.9122, device='cuda:0')
Batch Loss: 152.45376306958497 Accuracy: tensor(0.9121, device='cuda:0')
Batch Loss: 171.01855508238077 Accuracy: tensor(0.9141, device='cuda:0')
Batch Loss: 193.9484755024314 Accuracy: tensor(0.9131, device='cuda:0')
Batch Loss: 216.70403468422592 Accuracy: tensor(0.9129, device='cuda:0')
Batch Loss: 238.19008122198284 Accuracy: tensor(0.9131, device='cuda:0')
Batch Loss: 264.37462246604264 Accuracy: tensor(0.9107, device='cuda:0')
Batch Loss: 284.804384553805 Accuracy: tensor(0.9118, device='cuda:0')
Batch Loss: 304.2415911965072 Accuracy: tensor(0.9128, dev

  0%|          | 0/1544 [00:00<?, ?it/s]

Batch Loss: 20.762195078656077 Accuracy: tensor(0.9194, device='cuda:0')
Batch Loss: 39.86796177737415 Accuracy: tensor(0.9206, device='cuda:0')
Batch Loss: 59.47390894964337 Accuracy: tensor(0.9225, device='cuda:0')
Batch Loss: 78.41287332121283 Accuracy: tensor(0.9214, device='cuda:0')
Batch Loss: 95.06628671195358 Accuracy: tensor(0.9241, device='cuda:0')
Batch Loss: 112.4264778578654 Accuracy: tensor(0.9256, device='cuda:0')
Batch Loss: 132.9744677087292 Accuracy: tensor(0.9253, device='cuda:0')
Batch Loss: 152.62602995615453 Accuracy: tensor(0.9252, device='cuda:0')
Batch Loss: 170.88270287495106 Accuracy: tensor(0.9263, device='cuda:0')
Batch Loss: 188.1384557140991 Accuracy: tensor(0.9266, device='cuda:0')
Batch Loss: 206.9034796440974 Accuracy: tensor(0.9261, device='cuda:0')
Batch Loss: 230.55802951939404 Accuracy: tensor(0.9246, device='cuda:0')
Batch Loss: 249.5326071344316 Accuracy: tensor(0.9247, device='cuda:0')
Batch Loss: 268.96254677325487 Accuracy: tensor(0.9242, devi

In [ ]:
losses, accuracies

([547.208907250315,
  423.5157474204898,
  383.3380469288677,
  337.4223535428755,
  295.6810537548736],
 [tensor(0.8584, device='cuda:0'),
  tensor(0.8882, device='cuda:0'),
  tensor(0.8997, device='cuda:0'),
  tensor(0.9126, device='cuda:0'),
  tensor(0.9241, device='cuda:0')])

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "naver_map_model.pt") 

In [ ]:
# 모델 불러오기
model.load_state_dict(torch.load("./naver_map_model.pt"))

<All keys matched successfully>

# TEST

## Evaluation

In [ ]:
# 테스트 데이터셋 정확도 확인하기

model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.type(torch.LongTensor)
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/1435 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy: tensor(0.9127, device='cuda:0')


In [ ]:
# 리뷰 텍스트 predicted label(0,1) 확인

def sentences_predict(sent):
    tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
    model.eval()
    tokenized_sent = tokenizer(
            sent,
            return_tensors="pt",
            truncation=True,
            add_special_tokens=True,
            max_length=256
    )
    tokenized_sent.to(device)
    
    with torch.no_grad():# gradient 계산 비활성화
        outputs = model(
            input_ids=tokenized_sent['input_ids'],
            attention_mask=tokenized_sent['attention_mask'],
            token_type_ids=tokenized_sent['token_type_ids']
            )
        
    logits = outputs[0] 
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits) 
    return result

In [ ]:
sentences_predict('맛없음')

0

In [ ]:
sentences_predict('생각보다 맛있고, 사장님이 친절해요')

1

## Test Result

In [ ]:
score = [] # label - score

total_len = len(test_pd)

for cnt, review in enumerate(test_pd['리뷰']): 
  pred = sentiment_classifier(review) 
  score.append(pred) 
  print(cnt, '개 문장 분류 완료')

0 개 문장 분류 완료
1 개 문장 분류 완료
2 개 문장 분류 완료
3 개 문장 분류 완료
4 개 문장 분류 완료
5 개 문장 분류 완료
6 개 문장 분류 완료
7 개 문장 분류 완료
8 개 문장 분류 완료
9 개 문장 분류 완료
10 개 문장 분류 완료
11 개 문장 분류 완료
12 개 문장 분류 완료
13 개 문장 분류 완료
14 개 문장 분류 완료
15 개 문장 분류 완료
16 개 문장 분류 완료


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
22929 개 문장 분류 완료
22930 개 문장 분류 완료
22931 개 문장 분류 완료
22932 개 문장 분류 완료
22933 개 문장 분류 완료
22934 개 문장 분류 완료
22935 개 문장 분류 완료
22936 개 문장 분류 완료
22937 개 문장 분류 완료
22938 개 문장 분류 완료
22939 개 문장 분류 완료
22940 개 문장 분류 완료
22941 개 문장 분류 완료
22942 개 문장 분류 완료
22943 개 문장 분류 완료
22944 개 문장 분류 완료
22945 개 문장 분류 완료
22946 개 문장 분류 완료
22947 개 문장 분류 완료
22948 개 문장 분류 완료
22949 개 문장 분류 완료
22950 개 문장 분류 완료
22951 개 문장 분류 완료
22952 개 문장 분류 완료
22953 개 문장 분류 완료
22954 개 문장 분류 완료
22955 개 문장 분류 완료
22956 개 문장 분류 완료
22957 개 문장 분류 완료
22958 개 문장 분류 완료
22959 개 문장 분류 완료
22960 개 문장 분류 완료
22961 개 문장 분류 완료
22962 개 문장 분류 완료
22963 개 문장 분류 완료
22964 개 문장 분류 완료
22965 개 문장 분류 완료
22966 개 문장 분류 완료
22967 개 문장 분류 완료
22968 개 문장 분류 완료
22969 개 문장 분류 완료
22970 개 문장 분류 완료
22971 개 문장 분류 완료
22972 개 문장 분류 완료
22973 개 문장 분류 완료
22974 개 문장 분류 완료
22975 개 문장 분류 완료
22976 개 문장 분류 완료
22977 개 문장 분류 완료
22978 개 문장 분류 완료
22979 개 문장 분류 완료
22980 개 문장 분류 완료
22981 개 문장 분류 완료
22982 개 문장 분류 완료
22983 개 문장 분류 완료
22984 개 문장 분류 완료
22985 개 문장 분

In [ ]:
score

[[{'label': 'LABEL_1', 'score': 0.9962452054023743}],
 [{'label': 'LABEL_1', 'score': 0.9959371089935303}],
 [{'label': 'LABEL_0', 'score': 0.8869326114654541}],
 [{'label': 'LABEL_1', 'score': 0.9939180612564087}],
 [{'label': 'LABEL_0', 'score': 0.7629438042640686}],
 [{'label': 'LABEL_0', 'score': 0.9975324869155884}],
 [{'label': 'LABEL_1', 'score': 0.6353541016578674}],
 [{'label': 'LABEL_1', 'score': 0.9916336536407471}],
 [{'label': 'LABEL_1', 'score': 0.9959611296653748}],
 [{'label': 'LABEL_1', 'score': 0.99594646692276}],
 [{'label': 'LABEL_1', 'score': 0.9955370426177979}],
 [{'label': 'LABEL_1', 'score': 0.9925289750099182}],
 [{'label': 'LABEL_1', 'score': 0.868021547794342}],
 [{'label': 'LABEL_1', 'score': 0.9960891008377075}],
 [{'label': 'LABEL_1', 'score': 0.9942991733551025}],
 [{'label': 'LABEL_1', 'score': 0.9955588579177856}],
 [{'label': 'LABEL_1', 'score': 0.9957654476165771}],
 [{'label': 'LABEL_1', 'score': 0.9952123165130615}],
 [{'label': 'LABEL_1', 'score':

In [ ]:
test_pd['predicted'] = 0 # label(예측 결과): 1(긍정) / 0(부정)
score_list = [] # score 

for i in range(len(score)): 
    test_pd['predicted'][i] = int(score[i][0].get('label')[-1])
    score_list.append(float(score[i][0].get('score')))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
test_pd

,리뷰,평점,라벨링,predicted
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,1,1
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,1,1
2,좋아요,5.0,1,0
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,1,1
4,귀엽다,3.0,0,0
...,...,...,...,...
27924,생각보다 그냥 그래욤,3.5,0,0
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,1,1
27926,친절해요,5.0,1,0
27927,좋아요,5.0,1,0


In [ ]:
score_df = pd.DataFrame({'score':score_list})
final_test_pd = pd.concat([test_pd,score_df],axis=1)
final_test_pd

,리뷰,평점,라벨링,predicted,score
0,육수와 비빔양념 너무 맛있네요~이모님들도 친절하구요~,5.0,1,1,0.996245
1,스탭들 대박친절해요\n매운뼈구이 강력추천\n매콤하니 핵존맛탱~,5.0,1,1,0.995937
2,좋아요,5.0,1,0,0.886933
3,맛있게 잘먹고 갑니다. 아기가 있어 염도조절도 물어봐주고 고맙습니다.,5.0,1,1,0.993918
4,귀엽다,3.0,0,0,0.762943
...,...,...,...,...,...
27924,생각보다 그냥 그래욤,3.5,0,0,0.994940
27925,가격. 맛.서비스. 모두만족!! 메뉴 다먹어봤는데 고사리육계장 해장국추천요! 아기의...,5.0,1,1,0.992844
27926,친절해요,5.0,1,0,0.727375
27927,좋아요,5.0,1,0,0.886933


In [ ]:
# 결과 저장
final_test_pd.to_csv("./final_result.csv", encoding = 'utf-8-sig') 

In [ ]:
# 전체 결과 정확도 

from sklearn.metrics import classification_report
print(classification_report(final_test_pd['predicted'], final_test_pd['라벨링'])) 

              precision    recall  f1-score   support

           0       0.82      0.39      0.53      7379
           1       0.82      0.97      0.89     20550

    accuracy                           0.82     27929
   macro avg       0.82      0.68      0.71     27929
weighted avg       0.82      0.82      0.79     27929

